In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 6.3 MB/s 


In [ ]:
import pandas as pd
import re
import string
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud, STOPWORDS
from textwrap import wrap
# For visualizations
import matplotlib.pyplot as plt
import PyPDF2 
import pickle
import random
import string
from  itertools import chain

# 1. Loading Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/Movie_quotes.csv")
df.head()

,text
0,They do not!
1,They do to!
2,I hope so.
3,She okay?
4,Let's go.


In [ ]:
df.dropna(subset=['text'],inplace = True)
df.isnull().sum()

text    0
dtype: int64

In [ ]:
movie_quotes = list(df['text'])
movie_quotes

['They do not!',
 'They do to!',
 'I hope so.',
 'She okay?',
 "Let's go.",
 'Wow',
 "Okay -- you're gonna need to learn how to lie.",
 'No',
 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'Like my fear of wearing pastels?',
 'The "real you".',
 'What good stuff?',
 "I figured you'd get to the good stuff eventually.",
 'Thank God!  If I had to hear one more story about your coiffure...',
 "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'What crap?',
 'do you listen to this crap?',
 'No...',
 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'You always been this selfish?',
 'But',
 "Then that's all you had to say.",
 'Well, no...',
 "You never wanted to go out with 'me, did you?",
 'I was?',
 'I looked for you back at the party, but you always seemed to be "occupied".',
 'Tons',
 'Have fun tonight?',
 'I believe we share an art instructor',
 'You know Chastity?',
 '

#2. Preprocessing data

In [ ]:
# initializing punctuations string
punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''

In [ ]:
processed_movie_quotes = []
# Removing punctuations in string
# Using loop + punctuation string
for test_str in movie_quotes:
  for ele in test_str:
      if ele in punc:
          test_str = test_str.replace(ele, "")
  processed_movie_quotes.append(test_str.lower())


processed_movie_quotes


['they do not',
 'they do to',
 'i hope so',
 'she okay',
 "let's go",
 'wow',
 "okay  you're gonna need to learn how to lie",
 'no',
 "i'm kidding  you know how sometimes you just become this persona  and you don't know how to quit",
 'like my fear of wearing pastels',
 'the real you',
 'what good stuff',
 "i figured you'd get to the good stuff eventually",
 'thank god  if i had to hear one more story about your coiffure',
 "me  this endless blonde babble i'm like boring myself",
 'what crap',
 'do you listen to this crap',
 'no',
 "then guillermo says if you go any lighter you're gonna look like an extra on 90210",
 'you always been this selfish',
 'but',
 "then that's all you had to say",
 'well no',
 "you never wanted to go out with 'me did you",
 'i was',
 'i looked for you back at the party but you always seemed to be occupied',
 'tons',
 'have fun tonight',
 'i believe we share an art instructor',
 'you know chastity',
 'looks like things worked out tonight huh',
 'hi',
 "who kn

#3. Loading required files

  -->list of contractions <br>
  -->list of common homophones

In [ ]:
article  = ["a","an","the"]



with open('/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/Contractions.pkl', 'rb') as f:
  contractions = pickle.load(f)



with open('/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/Homophones_list.pkl', 'rb') as f:
  homophones = pickle.load(f)

In [ ]:
contractions_list = []
for each in contractions:
  new_each = each.replace("’","'").lower()
  contractions_list.append(new_each)

In [ ]:
contractions_list

["aren't",
 "there's",
 "can't",
 "they'd",
 "couldn't",
 "they'll",
 "didn't",
 "they're",
 "doesn't",
 "they've",
 "don't",
 "we'd",
 "hadn't",
 "we're",
 "hasn't",
 "we've",
 "haven't",
 "weren't",
 "he'd",
 "what'll",
 "he'll",
 "what're",
 "he's",
 "what's",
 "i'd",
 "what've",
 "i'll",
 "where's",
 "i'm",
 "who'd",
 "i've",
 "who'll",
 "isn't",
 "who're",
 "let's",
 "who's",
 "mightn't",
 "who've",
 "mustn't",
 "won't",
 "shan't",
 "wouldn't",
 "she'd",
 "you'd",
 "she'll",
 "you'll",
 "she's",
 "you're",
 "shouldn't",
 "you've",
 "that's",
 "“it's.”",
 "“it's,”",
 "it's"]

In [ ]:
all_homophones = []
homophones_list = []
for each in homophones:
    each_set = each.lower().split("/")
    homophones_list.append(each_set)


homophones_list

[['abel', 'able'],
 ['accede', 'exceed'],
 ['accept', 'except'],
 ['acclamation', 'acclimation'],
 ['acts', 'ax'],
 ['ad', 'add'],
 ['adam', 'atom'],
 ['addition', 'edition'],
 ['adds', 'adz', 'ads'],
 ['adduce', 'educe'],
 ['adherence', 'adherents'],
 ['adieu', 'ado'],
 ['ads', 'adds', 'adz'],
 ['aerial', 'ariel'],
 ['affect', 'effect'],
 ['affected', 'effected'],
 ['affects', 'effects'],
 ['afterward', 'afterword'],
 ['aid', 'aide'],
 ['ail', 'ale'],
 ['air', 'heir', 'err'],
 ['aisle', 'isle', 'i’ll'],
 ['ale', 'ail'],
 ['all ready', 'already'],
 ['all together', 'altogether'],
 ['all ways', 'always'],
 ['all', 'awl'],
 ['allowed', 'aloudallude', 'elude'],
 ['alluded', 'eluded'],
 ['allusion', 'illusion'],
 ['allusive', 'elusive', 'illusive'],
 ['aloud', 'allowed'],
 ['already', 'all ready'],
 ['altar', 'alter'],
 ['altogether', 'all together'],
 ['always', 'all ways'],
 ['amend', 'emend'],
 ['analyst', 'annalist'],
 ['ant', 'aunt'],
 ['apatite', 'appetite'],
 ['apprise', 'apprize'],

#4. Preparing data using perbutations

In [ ]:
def find_homophone_word(word):
    new_word = ""
    for sublist in homophones_list:
        if word in sublist:
           new_word = random.choice([ele for ele in sublist if ele != word])
          #  print("NEW WORD-->",new_word)
           return new_word


In [ ]:
def create_dataset(movie_quotes):
    i=0
    original_quotes_list = []
    modified_quotes_list = []
    for each_quote in movie_quotes:
      i+=1

      # print(each_quote)
      modified_quote = ""

      for each_word in each_quote.split():
        
        # print("EACH WORD-->",each_word)
        foo = [1,0,0,0]
        flag = random.choice(foo)

        if flag == 1 and each_word.lower() in article:
          # print("Article")
          pass


        elif flag == 1 and each_word.lower() in chain(*homophones_list):
          new_word = find_homophone_word(each_word.lower())
          modified_quote+= (new_word + " ")


        elif flag == 1 and each_word in contractions_list:
          # print("Contractions")
          new_word = each_word.split("'")[0]
          modified_quote+= (new_word + " ")


        else:
          modified_quote += (each_word + " ")

      original_quotes_list.append(each_quote)
      modified_quotes_list.append(modified_quote)

    return [original_quotes_list,modified_quotes_list]

In [ ]:
len(processed_movie_quotes)

304446

In [ ]:

def increase_dataset():

  global final_df
  
  results = create_dataset(processed_movie_quotes)
  original,modified  = results[0],results[1]
  df_new = pd.DataFrame(list(zip(original, modified)),               columns =['original', 'modified'])
  final_df = final_df.append(df_new)

  final_df= final_df.drop_duplicates()
  print(final_df.shape[0])
  return final_df.shape[0]

In [ ]:
final_df = pd.DataFrame(columns = ['original', 'modified'])
while True:
    
    total_datapoints = increase_dataset()
    if(total_datapoints> 500000):
      break

print(final_df.shape)

265442
448756
606586
(606586, 2)


In [ ]:
final_df.to_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/data_500k.csv",index = False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/data_500k.csv")

In [ ]:
df.head(30)

,original,modified
0,they do not,they due not
1,they do to,they do too
2,i hope so,i hope so
3,she okay,she okay
4,let's go,let's go
5,wow,wow
6,okay you're gonna need to learn how to lie,okay you're gonna need to learn how too lie
7,no,no
8,i'm kidding you know how sometimes you just b...,i'm kidding you know how sometimes ewe just be...
9,like my fear of wearing pastels,like my fear of wearing pastels


In [ ]:
df.tail(30)

,original,modified
606556,tomorrow we will continue our advance on ulund...,tomorrow we will continue hour advance on ulun...
606557,intelligence sources of your own did it not oc...,intelligence sources of you’re own did it not ...
606558,i have received intelligence from sources of m...,i have received intelligence from sources of m...
606559,you intended to bring your reserves across the...,you intended to bring you’re reserves across r...
606560,my lord this list was prepared for you i don ...,my lord this list was prepared for you i don '...
606561,excellent thank you give them to crealock wou...,excellent thank you give them two crealock wou...
606562,i thought it might be more effective to find s...,i thought it might be more effective to find s...
606563,oh indeed crealock we should see that colonel ...,owe indeed crealock we should see that colonel...
606564,they come well recommended do they durnford,they come well recommended dew they durnford
606565,sikali horse my lord christians all i know eac...,sikali hoarse my lord christians awl i know ea...
